In [67]:
import sys
sys.path.append('..')

import numpy as np
import pickle
import matplotlib.pyplot as plt
import os

import torch

from utils import *
TensorType = torch.DoubleTensor
from nmgp_dsvi import *
from sklearn import preprocessing
import statsmodels.api as sm
import time

In [62]:
def time2timestamp(x, rate):
    return np.round(x * rate).astype(int)


def load_ECoG(data_file, n_channel=None, channel_indexes=None, do_plot_raw_data=True, time_start=14., time_stop=16., rate=400.):
    data = "ECoG"
    if not os.path.exists("../../res/{}".format(data)):
        os.mkdir("../../res/{}".format(data))
    # Default setting
    times_test = np.arange(160) / rate
    # Upload Data
    time_trials = None
    if data_file == "data_R32_B7.pickle":
        data_loc = "../../data/ECoG/R32_B7_Hilb_54bands_ECoG_high_gamma.pickle"
        with open(data_loc, "rb") as res:
            times, band_resps, time_trials = pickle.load(res)
    if data_file == "data_R32_B8.pickle":
        data_loc = "../../data/ECoG/R32_B8_Hilb_54bands_ECoG_high_gamma.pickle"
        with open(data_loc, "rb") as res:
            times, band_resps = pickle.load(res)
    # take subsamples
    time_interval_name = "{}s_{}s".format(int(time_start), int(time_stop))
    time_interval = np.linspace(time_start, time_stop, num=int((time_stop - time_start) * (rate)), endpoint=False)
    timestamps_interval = time2timestamp(time_interval, rate)
    if data_file == "data_R32_B7.pickle":
        time_range = np.array([time_interval[0], time_interval[-1]])
        time_trials = extract_trails(time_trials, time_range)
    times = times[timestamps_interval]
    band_resps = band_resps[timestamps_interval]
    # import pdb; pdb.set_trace()

    N, M = band_resps.shape
    X_list = list()
    Y_list = list()
    for i in range(M):
        X_list.append(np.arange(N))
        Y_list.append(preprocessing.scale(band_resps[:, i]))
    # sample channels
    X_list, Y_list, Xt_list, Yt_list = create_datasets(X_list, Y_list, channel_indexes=channel_indexes)
    if n_channel is None:
        n_channel = len(X_list)
    n_dims = n_channel
    pre_name = "pred_data{}".format(n_channel)
    # convert data format
    X_train_origin_list = [x[:, None] for x in X_list]
    X_test_origin_list = [x[:, None] for x in Xt_list]
    Y_train_origin_list = [y[:, None] for y in Y_list]
    Y_test_origin_list = [y[:, None] for y in Yt_list]

    X_train_list = X_train_origin_list[:n_dims]
    X_test_list = X_test_origin_list[:n_dims]
    Y_train_list = Y_train_origin_list[:n_dims]
    Y_test_list = Y_test_origin_list[:n_dims]

    X_train_vec = np.concatenate(X_train_list)
    X_test_vec = np.concatenate(X_test_list)
    Y_train_vec = np.concatenate(Y_train_list)
    Y_test_vec = np.concatenate(Y_test_list)

    train_index = np.concatenate([np.ones_like(Y_train_list[i]) * i for i in range(n_dims)]).astype(int)
    test_index = np.concatenate([np.ones_like(Y_test_list[i]) * i for i in range(n_dims)]).astype(int)
    t_max = np.max(np.concatenate(X_list))

    if do_plot_raw_data:
        n_illustration = 10
        gap=10
        fig = plt.figure()
        ax = fig.add_subplot(111)
        for i in range(n_illustration):
            ax.plot(X_train_list[i], Y_train_list[i] - gap*i, 'ko', ms=0.5)
        ax.legend(loc="upper left")
        plt.show()

        standard_band_resps = preprocessing.scale(band_resps, axis=0)
        standard_band_resps = standard_band_resps[:, channel_indexes]
        plot_resps(times, standard_band_resps, time_trials)
        plot_resps_fb(times, standard_band_resps, channel_indexes, data_directory="../../data/ECoG")

    return X_train_list, X_test_list, Y_train_list, Y_test_list, X_train_vec, X_test_vec, Y_train_vec, Y_test_vec, \
           train_index, test_index, t_max, rate, times_test, data, pre_name, time_interval_name


def extract_trails(time_trails, time_range):
    res_list = []
    for time_trail in time_trails:
        if ((time_trail[0] > time_range[0]) and (time_trail[0] < time_range[1])) or ((time_trail[1] > time_range[0]) and (time_trail[1] < time_range[1])):
            res_list.append([np.max([time_trail[0], time_range[0]]), np.min([time_trail[1], time_range[1]])])
    return np.stack(res_list)


def create_datasets(X_list, Y_list, test_channel_index=0, channel_indexes=None, train_rate=0.8, random=False):
    np.random.seed(22)
    N = Y_list[0].shape[0]
    M = len(Y_list)
    if random:
        train_index = np.random.choice(N, size=int(train_rate * N), replace=False)
    else:
        train_index = np.arange(int(train_rate * N))
    test_index = np.array(list(set(np.arange(N)) - set(train_index)))

    Xtrain_list = list()
    Ytrain_list = list()
    Xtest_list = list()
    Ytest_list = list()
    if channel_indexes is None:
        channel_indexes = np.arange(M)
    for i in channel_indexes:
        if i == channel_indexes[test_channel_index]:
            Xtrain_list.append(X_list[i][train_index])
            Ytrain_list.append(Y_list[i][train_index])
            Xtest_list.append(X_list[i][test_index])
            Ytest_list.append(Y_list[i][test_index])
        else:
            Xtrain_list.append(X_list[i])
            Ytrain_list.append(Y_list[i])
            Xtest_list.append(np.array([]))
            Ytest_list.append(np.array([]))
    return Xtrain_list, Ytrain_list, Xtest_list, Ytest_list


def plot_resps(times, resps, time_trials):
    fig = plt.figure()
    resp_mean = np.mean(resps, axis=1)
    resp_std = np.std(resps, axis=1)
    resp_upper, resp_lower = resp_mean + resp_std, resp_mean - resp_std
    plt.plot(times, resp_mean)
    plt.plot(times, np.stack([resp_lower, resp_upper], axis=1), linestyle='--', color='r')
    if time_trials is not None:
        for time_trail in time_trials:
            plt.vlines(time_trail[0], ymin=-2, ymax=10)
            plt.vlines(time_trail[1], ymin=-2, ymax=10)
    plt.show()
    plt.close(fig)


def plot_resps_fb(times, resps, channel_indexes, data_directory=None):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    # import pdb; pdb.set_trace()
    res = sm.graphics.fboxplot(resps.T, wfactor=1000., labels=channel_indexes, ax=ax)
    ax.set_xlabel("Time (second)", fontsize=18)
    ax.set_ylabel("Z-score", fontsize=18)
    ax.set_xticks(np.arange(times.shape[0], step=100))
    ax.set_xticklabels(np.arange(times.shape[0], step=100)/400.)
    ax.tick_params(labelsize=16)
    plt.tight_layout()
    fig.savefig(data_directory + "/functional_boxplot.png")
    plt.show()


In [2]:
data = "ECoG"

if not os.path.exists("../../res/{}".format(data)):
    os.mkdir("../../res/{}".format(data))

with open("../../data/ECoG/78_channel_indexes.pickle", "rb") as res:
    channel_indexes = pickle.load(res)
n_channels = channel_indexes.shape[0]
data_file = "data_R32_B7.pickle"
# data_file = "data_R32_B8.pickle"
time_start = 14.
time_stop = 16.
rate = 400.
X_train_list, X_test_list, Y_train_list, Y_test_list, X_train_vec, X_test_vec, Y_train_vec, Y_test_vec, \
train_index, test_index, t_max, rate, times_test, data, pre_name, time_interval_name = load_ECoG(data_file, time_start=time_start,
    time_stop=time_stop, rate=rate, channel_indexes=channel_indexes)

do_plot_raw_data = True

if do_plot_raw_data:
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.set_title('Output 1')
    ax1.plot(X_train_list[0], Y_train_list[0],'ko',mew=1.5,label='Train set')
    ax1.plot(X_test_list[0], Y_test_list[0],'ro',mew=1.5,label='Test set')
    ax1.legend()
    plt.show()

In [3]:
# Sparse GPR
import GPy

K = GPy.kern.Exponential(1)
m1 = GPy.models.SparseGPRegression(X_train_list[0], Y_train_list[0], kernel=K.copy(), num_inducing=100)
m1.optimize(messages=True)

ts = time.time()
est_Y_test = m1.predict(X_test_list[0])[0]
print("testing costs {}s".format(time.time() - ts))

print(m1)

rmse_test = np.sqrt(np.mean((est_Y_test - np.vstack(Y_test_list)) ** 2))
print(rmse_test)

# print(est_Y_test)
# print(np.vstack(Y_test_list))

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.set_title('Output 1')
ax1.plot(X_test_list[0], est_Y_test,'rx',mew=1.5, label='Prediction set')
ax1.plot(X_test_list[0], np.vstack(Y_test_list),'kx',mew=1.5, label='test set')
ax1.legend()
plt.show()

In [4]:
# Sparse Corregionalization Model
import GPy

n_dims=25
K = GPy.kern.Exponential(1)
icm = GPy.util.multioutput.ICM(input_dim=1, num_outputs=n_dims, kernel=K)
m2 = GPy.models.SparseGPCoregionalizedRegression(X_train_list,Y_train_list,kernel=icm,num_inducing=100)
m2['.*Exponential.var'].constrain_fixed(1.) #For this kernel, B.kappa encodes the variance now.
print("start")
print(m2.log_likelihood())
m2.optimize(messages=True)

Xt_vecs = np.vstack([np.hstack([X_test_list[i], np.ones_like(X_test_list[i])*i]) for i in range(n_dims)])
noise_dictt = {'output_index':Xt_vecs[:,1].astype(int)}
est_Y_test = m2.predict(Xt_vecs, Y_metadata=noise_dictt)[0]

print(m2)

rmse_test = np.sqrt(np.mean((est_Y_test - np.vstack(Y_test_list)) ** 2))
print(rmse_test)